# **Homework 9: Variational Autoencoders**

## **About**

### **Due**

Monday 4/22/19, 11:59 PM CST

### **Goal**

This homework focuses on creating variational autoencoders applied to the MNIST dataset.

## Dev Environment
### Working on Google Colab
You may choose to work locally or on Google Colaboratory. You have access to free compute through this service. 
1. Visit https://colab.research.google.com/drive 
2. Navigate to the **`Upload`** tab, and upload your `HW10.ipynb`
3. Now on the top right corner, under the `Comment` and `Share` options, you should see a `Connect` option. Once you are connected, you will have access to a VM with 12GB RAM, 50 GB disk space and a single GPU. The dropdown menu will allow you to connect to a local runtime as well.

**Notes:** 
* **If you do not have a working setup for Python 3, this is your best bet. It will also save you from heavy installations like `tensorflow` if you don't want to deal with those.**
* ***There is a downside*. You can only use this instance for a single 12-hour stretch, after which your data will be deleted, and you would have redownload all your datasets, any libraries not already on the VM, and regenerate your logs**.


### Installing PyTorch and Dependencies

The instructions for installing and setting up PyTorch can be found at https://pytorch.org/get-started/locally/. Make sure you follow the instructions for your machine. For any of the remaining libraries used in this assignment:
* We have provided a `hw8_requirements.txt` file on the homework web page. 
* Download this file, and in the same directory you can run `pip3 install -r hw8_requirements.txt`

Check that PyTorch installed correctly by running the following:

In [1]:
import torch
torch.rand(5, 3)

tensor([[0.6912, 0.8272, 0.6919],
        [0.5361, 0.1642, 0.2506],
        [0.4831, 0.4717, 0.1631],
        [0.0912, 0.7848, 0.5164],
        [0.4380, 0.0152, 0.8380]])

The output should look something like

```python
tensor([[0.3380, 0.3845, 0.3217],
        [0.8337, 0.9050, 0.2650],
        [0.2979, 0.7141, 0.9069],
        [0.1449, 0.1132, 0.1375],
        [0.4675, 0.3947, 0.1426]])
```

### Let's get started with the assignment.

## **Instructions**

### **Part 1 - Datasets and Dataloaders**

This part of the assignment is similar to HW 8. 

**Create a directory named hw9_data with the following command.**

In [2]:
!mkdir hw9_data

mkdir: hw9_data: File exists



**Now use `torch.datasets.MNIST` to load the Train and Test data into `hw9_data`.** 
* ** Use the directory you created above as the `root` directory for your datasets**
* ** Populate the `transformations` variable with any transformations you would like to perform on your data.** (Hint: You will need to do at least one)
* **Pass your `transformations` variable to `torch.datasets.MNIST`. This allows you to perform arbitrary transformations to your data at loading time.**

In [3]:
from torchvision import datasets, transforms

transformations = transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1000,), (0.3000,))
                             ])

mnist_train = datasets.MNIST(root='./hw9_data', train=True, download=True, transform=transformations) 
mnist_test = datasets.MNIST(root='./hw9_data', train=False, download=True, transform=transformations) 

**Any file in our dataset will now be read at runtime, and the specified transformations we need on it will be applied when we need it.**. 

We could iterate through these directly using a loop, but this is not idiomatic. PyTorch provides us with this abstraction in the form of `DataLoaders`. The module of interest is `torch.utils.data.DataLoader`. 

`DataLoader` allows us to do lots of useful things
* Group our data into batches
* Shuffle our data
* Load the data in parallel using `multiprocessing` workers

**Use `DataLoader` to create a loader for the training set and one for the testing set**
* **Use a `batch_size` of 32 to start, you may change it if you wish.**
* **Set the `shuffle` parameter to `True`.** 

**Check that the data was loaded successfully before proceeding to the next sections. **

In [4]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=32, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                           batch_size=len(mnist_test), 
                                           shuffle=False)

random_seed = 1
log_interval= 100
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

## **Part 2 - Encoder and Decoders (0 points)**

In this section we will be creating the encoder and decoder for our variational autoencoder (VAE). 

You can take a look at the following to understand how VAE's work. 

*   https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf
*  http://kvfrans.com/variational-autoencoders-explained/
*  https://jmetzen.github.io/2015-11-27/vae.html

VAEs work around a latent space who's dimension can be chosen by us. We will leave this as a parameter for the Encoder and Decoder classes that you will have to populate. 

Feel free to use any network architecture that you wish. Try simpler network structures like a few linear layers before trying anything more complicated. 

### For the Encoder:

*   **Finish the __init__() function.**
*  **Finish the forward() function.** 
*  **Assume that input to forward, x, is of shape (batch_size, 28,28)**
*  **forward() should return two tensors of size latent_dim like a standard encoder of a VAE**
* **One of the tensors should correspond to the mean of the encoding and the other tensor should correspond to the variance. In practice, it is easier to model the output as the log of the variance (logvar) and we will too**

### For the Decoder:

*   **Finish the __init__() function.**
*  **Finish the forward() function.** 
*  **Assume that input to forward, x, is of shape (batch_size, latent_dim)**
*  **forward() should return a tensor of shape (batch_size, 28,28)**
* **Make sure that the output lies in the same range as the input to the encoder (Hint: Sigmoid?)**







In [5]:
from torch import nn
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        # rectified linear unit layer from 400 to 400
        self.relu = nn.ReLU()
        self.fc21 = nn.Linear(400, latent_dim)
        self.fc22 = nn.Linear(400, latent_dim)
  
    def forward(self, x):
        # h1 is [32, 400]
        x = x.view(-1, 784)
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(latent_dim, 400)
        self.fc4 = nn.Linear(400, 784)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,x):
        h3 = self.relu(self.fc3(x))
        return self.sigmoid(self.fc4(h3))

## **Part 3: Training and loss functions** (0 points)

Recall that the encoder outputs the mean (mu) and the log of the variance (logvar). This implies that the latent vector of the input image follows a gaussian distribution with mean (mu) and standard deviation (e^[0.5\*logvar]). To decode this information, the decoder needs a sample from this distribution. 

**Complete the sample function to generate these samples **

In [6]:
def sample(mu, logvar):
    std = logvar.mul(0.5).exp_()
    eps = Variable(std.data.new(std.size()).normal_())
    return eps.mul(std).add_(mu) #[32,20 ]


We also need to create the loss function. Assume that x are your input images and x_hat are your reconstructions of these input images, complete the following loss for a VAE. (Hint: You will need to use mu and logvar as well)

In [7]:
def vae_loss(x, x_hat, mu, logvar):
    # MSE LOSS + KL DIVERGENCE
    MSE = ((x_hat - x.view(-1, 784))**2).mean()
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    KLD /= 32 * 784 #batch_size*28*28

    return MSE + KLD

In the following we will instantiate an Encoder and Decoder with a latent dimension of 32.

We also define a single optimizer that optimizes the parameters of both the Encoder and the Decoder together. Feel free to use any optimizer of your choice. 

In [8]:
from torch import optim
from torchvision.utils import save_image
from torch.autograd import Variable

encoder = Encoder(32)
decoder = Decoder(32)
params = list(encoder.parameters())+list(decoder.parameters())
optimizer = optim.Adam(params, lr=1e-3)

Complete the train function that takes input encoder, decoder, train_loader, optimizer, and number of epochs you wish to train your model for.

Training will involve:

1.   **One epoch is defined as a full pass of your dataset through your model. We choose the number of epochs we wish to train our model for.**
2.   **For each batch, use the encoder to generate the mu and logvar.**
3. **Sample a latent vector for each image in the batch and feed this to the decoder to generate the decoded images.**
4. **Calculate the loss function for this batch.**
5. **Now calculate the gradients for each parameter you are optimizing over. (Hint: Your loss function object can do this for you)**
6. **Update your model parameters (Hint: The optimizer comes in here)**
7. ** Set the gradients in your model to zero for the next batch.**



In [9]:
def train_internal(epoch, encoder, decoder, train_loader, optimizer):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()

        mu, logvar = encoder(data)
        z = sample(mu, logvar)
        decoded_images = decoder(z)
        loss = vae_loss(data, decoded_images, mu, logvar)

        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def train(encoder, decoder, train_loader, optimizer, num_epochs = 10):
    for epoch in range(1, num_epochs + 1):
        train_internal(epoch, encoder, decoder, train_loader, optimizer)
        

Finally call train with the relevant parameters.

Note : This function may take a while to complete if you're training for many epochs on a cpu. This is where it comes in handy to be running on Google Colab, or just have a GPU on hand.

In [10]:
train(encoder, decoder, train_loader, optimizer, num_epochs = 10)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.037833
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.023550
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.022840
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.020479
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.019192
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.019589
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.019545
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.019405
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.020577
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.019332
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.019211
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.019499
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.017950
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.018362
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.020028
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.019240
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.017536
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.019709
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.018028
====> Epoch: 1 Average loss: 0.0198
Train Epoch: 2 [0/60000

Train Epoch: 9 [25600/60000 (43%)]	Loss: 0.016356
Train Epoch: 9 [28800/60000 (48%)]	Loss: 0.018589
Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.016933
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.016563
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.017584
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.017692
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.018006
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.017234
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.019407
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.019254
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.016555
====> Epoch: 9 Average loss: 0.0177
Train Epoch: 10 [0/60000 (0%)]	Loss: 0.017159
Train Epoch: 10 [3200/60000 (5%)]	Loss: 0.017412
Train Epoch: 10 [6400/60000 (11%)]	Loss: 0.018007
Train Epoch: 10 [9600/60000 (16%)]	Loss: 0.017320
Train Epoch: 10 [12800/60000 (21%)]	Loss: 0.017364
Train Epoch: 10 [16000/60000 (27%)]	Loss: 0.017111
Train Epoch: 10 [19200/60000 (32%)]	Loss: 0.018279
Train Epoch: 10 [22400/60000 (37%)]	Loss: 0.017332
Train Epoch: 10

## **Part 4: Visualizing the VAE output** (90 points)

We will look at how well the codes produced by the VAE can be interpolated. **For this section we will only use the MNIST test set. **

To create an interpolation between two images A and B, we encode both these images and generate a sample code for each of them. We now consider 7 equally spaced points in between these two sample codes giving us a total of 9 points including the samples. We then decode these images to get interpolated images in between A and B.

Complete the interpolation function below that takes a pair of images A and B and returns 9 images. (You are free to use any data structure you want to return these images)

In [17]:
import matplotlib.pyplot as plt
from torchvision import utils
%matplotlib inline
import numpy as np
import cv2
import random, string

def visualizeInterpolation(A, B, encoder, decoder, num_interpolations):
    # Compute latent space projection
    mu_a, var_a = encoder(A)
    mu_b, var_b = encoder(B)
    sample_a = sample(mu_a, var_a)
    sample_b = sample(mu_b, var_b)

    vectors = []
    reconstructions = []
    #Linear interpolation
    alphaValues = np.linspace(0, 1, num_interpolations)
    for alpha in alphaValues:
        # Latent space interpolation
        vector = sample_a*(1-alpha) + sample_b*alpha
        reconstruction = decoder(vector)
        reconstructions.append(reconstruction)

    # Put final image together
    resultLatent = None
    
    for i in range(len(reconstructions)):
        reconstructedImage = reconstructions[i].detach().numpy()*255
        reconstructedImage = reconstructedImage.reshape([28,28])
        reconstructedImage = cv2.resize(reconstructedImage,(50,50))
        reconstructedImage = reconstructedImage.astype(np.uint8)
        resultLatent = reconstructedImage if resultLatent is None else np.hstack([resultLatent,reconstructedImage])

    return resultLatent


def create_interpolates(similar_pairs, encoder, decoder, name, num_interpolations):
    result = None
    for i in range(10):
        resultLatent = visualizeInterpolation(similar_pairs[i][0], similar_pairs[i][1], encoder, decoder, num_interpolations)
        result = resultLatent if result is None else  np.vstack([result,resultLatent])
        
    cv2.imwrite('Visuals/' + name + '.png', result)
    print('Image saved')
   

**For 10 pairs of MNIST test images of the same digit (1 pair for "0", 1 pair for "1", etc.), selected at random, compute the code for each image of the pair. Now compute 7 evenly spaced linear interpolates between these codes, and decode the result into images. Prepare a figure showing this interpolate. Lay out the figure so each interpolate is a row. On the left of the row is the first test image; then the interpolate closest to it; etc; to the last test image. You should have a 10 rows (1 row per digit) and 9 columns (7 interpolates + 2 selected test images) of images. (45 points)**

In [18]:

similar_pairs = {}
for _, (x, y) in enumerate(test_loader):
    for i in range(len(y)):
        if y[i].item() not in similar_pairs:
            similar_pairs[y[i].item()] = []
        if len(similar_pairs[y[i].item()])<2:
            similar_pairs[y[i].item()].append(x[i])
  
    done = True
    for i in range(10):
        if i not in similar_pairs or len(similar_pairs[i])<2:
            done = False
  
    if done:
        break

# similar_pairs[i] contains two images indexed at 0 and 1 that have images of the digit i
create_interpolates(similar_pairs,encoder, decoder, 'Same digits interpolations', 9)

Image saved


**For 10 pairs of MNIST test images, selected at random, compute the code for each image of the pair. Now compute 7 evenly spaced linear interpolates between these codes, and decode the result into images. Prepare a figure showing this interpolate. Lay out the figure so each interpolate is a row. On the left of the row is the first test image; then the interpolate closest to it; etc; to the last test image. You should have a 10 rows and 9 columns of images. (45 points)**

In [20]:
random_pairs = {}
for _, (x, y) in enumerate(test_loader):
  # Make sure the batch size is greater than 20
    for i in range(10):
        random_pairs[i] = []
        random_pairs[i].append(x[2*i])
        random_pairs[i].append(x[2*i+1])
    break

# random_pairs[i] contains two images indexed at 0 and 1 that are chosen at random
create_interpolates(random_pairs,encoder, decoder, 'Different digits interpolations', 9)


Image saved
